In [ ]:
import pandas as pd 

from sklearn.model_selection import train_test_split

# Load pand split data
df = pd.read_csv(r'Data\IRMAS_features_2_sec.csv')
x = df.drop(columns=["file_name", "label"])
y = df["label"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) #80% Training, 20% testing

print(f"Training set size: {x_train.shape}")
print(f"Test set size: {x_test.shape}")

In [ ]:
import tensorflow as tf
from keras import layers, models
from sklearn.preprocessing import LabelEncoder

#Encode labels into numeric values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

#simple neural network model definition
model = models.Sequential([
    layers.Dense(128, input_shape=(x_train.shape[1],), activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax') #output layer
])
#Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the model
history = model.fit(x_train, y_train_encoded, epochs=10, batch_size=32, validation_split=0.2)

#Evaluate model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test_encoded)
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

#Model evaluation
y_pred = np.argmax(model.predict(x_test), axis=1) #predictions on test set
print(classification_report(y_test_encoded, y_pred, target_names=label_encoder.classes_)) #Generate calssification report
print(confusion_matrix(y_test_encoded, y_pred)) #Confusion matrix

In [ ]:
import matplotlib.pyplot as plt

#Plot Accuracy
plt.plot(history.history['accuracy'], label='Training Accuaracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#Plot Loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#Save trained model in HDF% formate
model.save('Instrument_model.h5')

# Save the label encoder classes (the instrument classes)
import numpy as np
np.save('Instrument_classes.npy', label_encoder.classes_)